# Coupled, check methane calibration

In [ ]:
import os

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch

In [ ]:
os.makedirs('../plots/', exist_ok=True)

In [ ]:
temp_ukesm = np.ones((165, 19)) * np.nan
for run in range(19):
    try:
        temp_ukesm[:, run] = pd.read_csv('../data/fredriksen2022/historical/UKESM1-0-LL_historical_r%di1p1f2_anomalies.txt' %(run+1), index_col=0)['tas'].values
    except FileNotFoundError:
        pass

In [ ]:
nc = Dataset('../data/folberth2022/anthropogenic_ch4_cmip6_ceds__tgpyr.nc')

In [ ]:
nc.variables['emissions_CH4'][:].squeeze()

In [ ]:
nc.variables['time']

In [ ]:
nc.variables['time'][-1]

In [ ]:
nc.variables['emissions_CH4'][12:].squeeze()

In [ ]:
ch4_annual = np.zeros(165)
for year in range(165):
    ch4_annual[year] = nc.variables['emissions_CH4'][12*year+12:12*year+24].squeeze().mean()

In [ ]:
nc_wl = Dataset('../data/folberth2022/bl593_am-ch4-wtlems.nc')
wetland_ch4 = nc_wl.variables['m01s50i304'][:,0,0]
nc_wl.close()
wetland_ch4_estimate = np.zeros(165)
wetland_ch4_estimate[1:] = wetland_ch4 - 164.5

In [ ]:
pl.plot(ch4_annual)

In [ ]:
scenarios = ['ssp245']

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
natural_forcing_df = pd.read_csv('../data/calibration/natural_forcing.csv', index_col=0)

In [ ]:
f = FAIR(ch4_method="Thornhill2021")

In [ ]:
configs = ['UKESM1-0-LL']
species, properties = read_properties()
species.remove('CO2 FFI')      # c-driven run
#species.remove('CO2 AFOLU')    # c-driven run
species.remove('Halon-1202')   # not in CMIP6 list of species
species.remove('Contrails')    # not modelled in UKESM, I think
species.remove('NOx aviation') # which renders this redundant
species.remove('Light absorbing particles on snow and ice')  # I believe not modelled in UKESM
del properties['CO2 FFI']
#del properties['CO2 AFOLU']
del properties['Halon-1202']
del properties['Contrails']
del properties['NOx aviation']
del properties['Light absorbing particles on snow and ice']
for specie in species:
    if properties[specie]['greenhouse_gas'] and specie!='CH4':
        properties[specie]['input_mode'] = 'concentration'
properties

In [ ]:
f.define_time(1850, 2015, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])

In [ ]:
f.fill_species_configs()

In [ ]:
f.fill_from_rcmip()

In [ ]:
# 1850 baselines
for specie in ['Sulfur', 'BC', 'OC', 'NOx', 'VOC', 'CO', 'NH3']:
    f.species_configs['baseline_emissions'].loc[dict(specie=specie)] = f.emissions.loc[dict(specie=specie, timepoints=1850.5, scenario='ssp245')]

for specie in ['CO2', 'CH4', 'N2O', 'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa',
    'HFC-32', 'HFC-365mfc', 'HFC-4310mee']:
    f.species_configs['baseline_concentration'].loc[dict(specie=specie)] = f.concentration.loc[dict(specie=specie, timebounds=1850, scenario='ssp245')]

    
f.species_configs['baseline_concentration'].loc[dict(specie='CH4')] = 808.249028523763
f.species_configs['baseline_concentration'].loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0

## rebase volcanic to zero mean
#f.forcing.loc[dict(specie='Volcanic')] = f.forcing.loc[dict(specie='Volcanic')] - f.forcing.loc[dict(specie='Volcanic', scenario='ssp245', config='UKESM1-0-LL')].mean()

# override RCMIP natural forcing
fill(f.forcing, natural_forcing_df.loc[1850:2015, 'solar'][:, None, None], specie='Solar')
fill(f.forcing, natural_forcing_df.loc[1850:2015, 'volcanic'][:, None, None], specie='Volcanic')

# override RCMIP methane emissios
fill(f.emissions, ch4_annual[:, None, None] + wetland_ch4_estimate[:, None, None], specie='CH4')

In [ ]:
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.concentration, f.species_configs['baseline_concentration'])

In [ ]:
ods = ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
        'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402']

hfc = ['CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
        'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
        'HFC-365mfc', 'HFC-4310mee']

In [ ]:
# Methane lifetime calibs here
f.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="CH4")] = +0.22/1023.2219696044921
f.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="NOx")] = -0.25/142.18364862292066
f.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="VOC")] = +0.11/166.74246925530488
f.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="Equivalent effective stratospheric chlorine")] = -0.049/1133.249639460932
f.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="N2O")] = -0.012/53.96694437662762
f.species_configs['lifetime_temperature_sensitivity'][:] = -0.0408
f.species_configs['unperturbed_lifetime'].loc[dict(specie="CH4")] = 8
f.species_configs['baseline_emissions'].loc[dict(specie="CH4")] = ch4_annual[0]

# UKESM specific ESM component calibrations including hand edits from 13:
f.species_configs['land_use_cumulative_emissions_to_forcing'].loc[dict(specie="CO2 AFOLU")] = 0.0003024895261521421
f.species_configs['forcing_scale'].loc[dict(specie="CH4")] = 1.11547955
f.species_configs['h2o_stratospheric_factor'].loc[dict(specie="CH4")] = 1.29652705e-01
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="CH4")] = 1.27049657e-04
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="Equivalent effective stratospheric chlorine")] = -0.00029119797470220245
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="N2O")] = 0.0007481397748679878
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="VOC")] = 0.0006596999582126578
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="NOx")] = 0.000984642055228785
f.species_configs['forcing_temperature_feedback'].loc[dict(specie="Ozone")] = -0.079  # Recalibrated feedback from 11b
f.species_configs['forcing_scale'].loc[dict(specie="N2O")] = 1.0845323023219764
f.species_configs['forcing_scale'].loc[dict(specie="CO2")] = 0.9935419214400296

for specie in species:
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie=specie)] = 0
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = -0.00447635
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = 0.05297455
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = -0.00781294
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='CH4')] = -5.09377882e-05
f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = -0.63667782
f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 0.03110931
f.species_configs['aci_shape'].loc[dict(specie='BC')] = 0.00221948
f.species_configs['aci_shape'].loc[dict(specie='OC')] = 0.00864984
f.species_configs['aci_shape'].loc[dict(specie='CH4')] = -3.54174562e-04

for specie in ods:
    f.species_configs['forcing_scale'].loc[dict(specie=specie)] = 1.33139609
for specie in hfc:
    f.species_configs['forcing_scale'].loc[dict(specie=specie)] = 0.25250486
    
f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 0.03110931 * 2
f.species_configs['forcing_efficacy'].loc[dict(specie="Aerosol-cloud interactions")] = 1.1

In [ ]:
f.run()

In [ ]:
pl.plot(f.timebounds, f.temperature[:, :, 0, 0])

In [ ]:
f.emissions.loc[dict(specie="VOC")].squeeze()

In [ ]:
pl.plot(f.alpha_lifetime.loc[dict(specie='CH4')].squeeze() * 8)

In [ ]:
nc1 = Dataset('../data/folberth2022/bl593_amgm-ch4sfcmolfrac.nc')

In [ ]:
ukesm_ch4 = nc1.variables['mass_fraction_of_methane_in_air'][:]
nc1.variables['time'][0]

In [ ]:
nc1.variables['time']

In [ ]:
nc1.variables['time'][0] / 360 / 24 + 1970
nc1.close()

In [ ]:
wetland_ch4_estimate = np.nanmean(temp_ukesm, axis=1) * 40

In [ ]:
g = FAIR(ch4_method="Thornhill2021")

In [ ]:
g.define_time(1850, 2015, 1)
g.define_scenarios(scenarios)
g.define_configs(configs)
g.define_species(species, properties)

In [ ]:
g.allocate()

In [ ]:
fill(g.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(g.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(g.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(g.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])

In [ ]:
g.fill_species_configs()

In [ ]:
g.fill_from_rcmip()

In [ ]:
# 1850 baselines
for specie in ['Sulfur', 'BC', 'OC', 'NOx', 'VOC', 'CO', 'NH3']:
    g.species_configs['baseline_emissions'].loc[dict(specie=specie)] = g.emissions.loc[dict(specie=specie, timepoints=1850.5, scenario='ssp245')]

for specie in ['CO2', 'CH4', 'N2O', 'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa',
    'HFC-32', 'HFC-365mfc', 'HFC-4310mee']:
    g.species_configs['baseline_concentration'].loc[dict(specie=specie)] = g.concentration.loc[dict(specie=specie, timebounds=1850, scenario='ssp245')]

    
g.species_configs['baseline_concentration'].loc[dict(specie='CH4')] = 808.249028523763
g.species_configs['baseline_concentration'].loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0

## rebase volcanic to zero mean
#f.forcing.loc[dict(specie='Volcanic')] = f.forcing.loc[dict(specie='Volcanic')] - f.forcing.loc[dict(specie='Volcanic', scenario='ssp245', config='UKESM1-0-LL')].mean()

# override RCMIP natural forcing
fill(g.forcing, natural_forcing_df.loc[1850:2015, 'solar'][:, None, None], specie='Solar')
fill(g.forcing, natural_forcing_df.loc[1850:2015, 'volcanic'][:, None, None], specie='Volcanic')

# override RCMIP methane emissios
fill(g.emissions, ch4_annual[:, None, None] + wetland_ch4_estimate[:, None, None], specie='CH4')

In [ ]:
initialise(g.forcing, 0)
initialise(g.temperature, 0)
initialise(g.cumulative_emissions, 0)
initialise(g.airborne_emissions, 0)
initialise(g.concentration, g.species_configs['baseline_concentration'])

In [ ]:
ods = ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
        'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402']

hfc = ['CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
        'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
        'HFC-365mfc', 'HFC-4310mee']

In [ ]:
# Methane lifetime calibs here
g.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="CH4")] = +0.22/1023.2219696044921
g.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="NOx")] = -0.25/142.18364862292066
g.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="VOC")] = +0.11/166.74246925530488
g.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="Equivalent effective stratospheric chlorine")] = -0.049/1133.249639460932
g.species_configs['ch4_lifetime_chemical_sensitivity'].loc[dict(specie="N2O")] = -0.012/53.96694437662762
g.species_configs['lifetime_temperature_sensitivity'][:] = -0.0408
g.species_configs['unperturbed_lifetime'].loc[dict(specie="CH4")] = 8
g.species_configs['baseline_emissions'].loc[dict(specie="CH4")] = ch4_annual[0]

# UKESM specific ESM component calibrations including hand edits from 13:
g.species_configs['land_use_cumulative_emissions_to_forcing'].loc[dict(specie="CO2 AFOLU")] = 0.0003024895261521421
g.species_configs['forcing_scale'].loc[dict(specie="CH4")] = 1.11547955
g.species_configs['h2o_stratospheric_factor'].loc[dict(specie="CH4")] = 1.29652705e-01
g.species_configs['ozone_radiative_efficiency'].loc[dict(specie="CH4")] = 1.27049657e-04
g.species_configs['ozone_radiative_efficiency'].loc[dict(specie="Equivalent effective stratospheric chlorine")] = -0.00029119797470220245
g.species_configs['ozone_radiative_efficiency'].loc[dict(specie="N2O")] = 0.0007481397748679878
g.species_configs['ozone_radiative_efficiency'].loc[dict(specie="VOC")] = 0.0006596999582126578
g.species_configs['ozone_radiative_efficiency'].loc[dict(specie="NOx")] = 0.000984642055228785
g.species_configs['forcing_temperature_feedback'].loc[dict(specie="Ozone")] = -0.079  # Recalibrated feedback from 11b
g.species_configs['forcing_scale'].loc[dict(specie="N2O")] = 1.0845323023219764
g.species_configs['forcing_scale'].loc[dict(specie="CO2")] = 0.9935419214400296

for specie in species:
    g.species_configs['erfari_radiative_efficiency'].loc[dict(specie=specie)] = 0
g.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = -0.00447635
g.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = 0.05297455
g.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = -0.00781294
g.species_configs['erfari_radiative_efficiency'].loc[dict(specie='CH4')] = -5.09377882e-05
g.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = -0.63667782
g.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 0.03110931
g.species_configs['aci_shape'].loc[dict(specie='BC')] = 0.00221948
g.species_configs['aci_shape'].loc[dict(specie='OC')] = 0.00864984
g.species_configs['aci_shape'].loc[dict(specie='CH4')] = -3.54174562e-04

for specie in ods:
    g.species_configs['forcing_scale'].loc[dict(specie=specie)] = 1.33139609
for specie in hfc:
    g.species_configs['forcing_scale'].loc[dict(specie=specie)] = 0.25250486
    
g.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 0.03110931 * 2
g.species_configs['forcing_efficacy'].loc[dict(specie="Aerosol-cloud interactions")] = 1.1

In [ ]:
g.run()

In [ ]:
pl.plot(f.timebounds, f.concentration[:, :, 0, 2], label = 'FaIR: wetland emissions prescribed')
pl.plot(g.timebounds, g.concentration[:, :, 0, 2], label = 'FaIR: 40TgCH4/K wetland feedback')
pl.plot(f.timebounds[1:-1], ukesm_ch4[:, 0, 0, 0], label = 'UKESM')
pl.legend()
pl.ylabel('ppb')
pl.savefig('../plots/CH4_emissions_driven.png')

In [ ]:
#pl.plot(wetland_ch4-165)

In [ ]:
#wetland_ch4[:20].mean()